In [40]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
import datacompy

folder_path = r'C:\Users\hoa.nd\Desktop\WAP\compare\report_3\result'

In [41]:
# Hiển thị danh sách column và số lượng column
def get_column_lists_and_counts(df_cognos, df_looker):
    columns_df_cognos = df_cognos.columns.tolist()
    columns_df_looker = df_looker.columns.tolist()
    num_columns_df_cognos = len(columns_df_cognos)
    num_columns_df_looker = len(columns_df_looker)
    print("Columns in df_cognos:", columns_df_cognos)
    print("Number of columns in df_cognos:", num_columns_df_cognos)
    print("Columns in df_looker:", columns_df_looker)
    print("Number of columns in df_looker:", num_columns_df_looker)
    return columns_df_cognos, num_columns_df_cognos, columns_df_looker, num_columns_df_looker

# Định nghĩa hàm để xóa các cột từ DataFrame
def drop_columns(df, columns_to_drop):
    df = df.drop(columns=columns_to_drop)
    return df

# Hàm Đổi các cột có nhãn (Ví dụ: Y -> Yes, N -> No)
def replace_values(df, columns, replacement):
    for column in columns:
        df[column] = df[column].replace(replacement)
    return df

#Định nghĩa hàm xóa dấu phẩy và convert cột sang kiểu Float để so sánh các cột số
def replace_comma_and_convert_to_float(df, column_names):
    for column_name in column_names:
        if column_name in df.columns:  # Kiểm tra xem cột có tồn tại trong DataFrame không
            if df[column_name].dtype == 'object' and df[column_name].str.contains(',').any():
                df[column_name] = df[column_name].str.replace(',', '', regex=True)
            df[column_name] = df[column_name].astype(float)


# Convert sang datetime
def convert_columns_to_datetime(df, columns):
    for column in columns:
        try:
            df[column] = pd.to_datetime(df[column], errors='coerce')
        except (KeyError, ValueError):
            print(f"Column '{column}' does not exist or cannot be converted to datetime.")

# fillna cho các cột chuỗi và số
def fillna_custom(df, string_columns, numeric_columns):
    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].fillna("No_Data")
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    return df

#Định nghĩa hàm trả về kiểu dữ liệu các column của DF
def check_and_print_column_data_types(df, name):
    data_types_df = df.dtypes
    print(f"Data types for DataFrame {name}:")
    print(data_types_df)

# KIỂM TRA CÁC COLUMN ĐÃ CHỌN CÓ TẠO RA ĐƯỢC TỔ HỢP KHÓA HAY KHÔNG
def check_columns_combination_uniqueness(df_cognos, df_looker, columns_to_check):
    def combine_columns(df):
        return df[columns_to_check].apply(lambda x: '-'.join(x.astype(str).str.upper()), axis=1)
    df_cognos_combined = combine_columns(df_cognos)
    df_looker_combined = combine_columns(df_looker)
    are_columns_combined_unique = df_cognos_combined.is_unique and df_looker_combined.is_unique
    if are_columns_combined_unique:
        result = True
    else:
        result = False
    return result

#ĐỊNH NGHĨA HÀM ĐỂ CHUYỂN SANG CHỮ IN HOA
def uppercase_columns(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.upper()
    return df

# ĐỊNH NGHĨA HÀM: TÌM VÀ CẮT DUOID (VD: _cognos, _looker)
def find_and_strip_columns_with_suffix(df, suffix):
    columns_with_suffix = [col for col in df.columns if col.endswith(suffix)]
    stripped_columns = [col.replace(suffix, '') for col in columns_with_suffix]
    return stripped_columns

# ĐỊNH NGHĨA HÀM: TÌM RA SỰ KHÁC BIỆT CỦA 2 DF (MISSING, DIFF) RÒI FILLNA
# Tìm sự khác biẹt
def create_diff_df(df_merged, columns):
    conditions = []
    for column in columns:
        data_cognos = df_merged[f"{column}_cognos"]
        data_looker = df_merged[f"{column}_looker"]
        if data_cognos.dtype== np.float64:
            data_cognos = data_cognos.round(2)
            data_looker = data_looker.round(2)
            condition = np.isclose(data_cognos, data_looker, atol=0.1)
        elif data_cognos.dtype== np.datetime64:
            rounded_cognos = data_cognos.dt.round('S')
            rounded_looker = data_looker.dt.round('S')
            condition = (rounded_cognos == rounded_looker)
        elif data_cognos.dtype== object:
            condition = (data_cognos.str.upper() == data_looker.str.upper())
        else:
            continue
        conditions.append(condition)
    combined_condition = np.all(conditions, axis=0)
    diff_df = df_merged[~combined_condition].copy()
    matching_df = df_merged[combined_condition].copy()
    diff_df = diff_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    matching_df = matching_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    return matching_df, diff_df


# LỌC RA PHẦN MISSING HAY DIFF
def filter_and_return_dataframes(df):
    df_missing_cognos = df.loc[df.filter(like='_cognos').eq('Not exists').all(axis=1)]
    df_missing_looker = df.loc[df.filter(like='_looker').eq('Not exists').all(axis=1)]
    df_remaining_diff = df.drop(df_missing_cognos.index).drop(df_missing_looker.index)
    return df_missing_cognos, df_missing_looker, df_remaining_diff.reset_index()

# TÁCH DF CỦA COGNOS VÀ LOOKER
def split_dataframe(df):
    cols_df_cognos = [col for col in df.columns if not col.endswith("_looker")]
    df_cognos_new = df[cols_df_cognos]
    cols_df_looker = [col for col in df.columns if not col.endswith("_cognos")]
    df_looker_new = df[cols_df_looker]
    return df_cognos_new, df_looker_new

# COMPARE VÀ LÊN MÀU CHO SỰ KHÁC BIẸT
def compare_and_style_data(diff_df, keys, suffix_cognos, suffix_looker):
    def highlight_cells(val):
        if val == 'Same':
            return ''
        else:
            return 'background-color: red; color: white;'
    common_prefixes = set(col.split(suffix_cognos)[0] for col in diff_df.columns[2:])
    for prefix in common_prefixes:
        cognos_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_cognos}')]
        looker_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_looker}')]
        for cognos_col, looker_col in zip(cognos_cols, looker_cols):
            column_name = f'Compare_{prefix}_(Cognos <> Looker)'
            data_type = diff_df[cognos_col].dtype
            if data_type == np.float64:
                diff_df[cognos_col] = diff_df[cognos_col].round(2)
                diff_df[looker_col] = diff_df[looker_col].round(2)
                condition = np.isclose(diff_df[cognos_col], diff_df[looker_col], atol=0.1)
            elif data_type == np.datetime64:
                rounded_cognos = diff_df[cognos_col].dt.round('S')
                rounded_looker = diff_df[looker_col].dt.round('S')
                condition = (rounded_cognos == rounded_looker)
            elif data_type == object:
                condition = (diff_df[cognos_col].str.upper() == diff_df[looker_col].str.upper())
            else:
                continue
            diff_df[column_name] = diff_df.apply(lambda row: f'({row[cognos_col]} <> {row[looker_col]})'
                                                if not condition[row.name]
                                                else 'Same', axis=1)
    selected_keys = keys + [col for col in diff_df.columns if col.startswith('Compare')]
    styled_df = diff_df[selected_keys].style.applymap(highlight_cells, subset=pd.IndexSlice[:, diff_df[selected_keys].columns.str.startswith('Compare_')])
    return styled_df


# ĐỊNH NGHĨA HÀM TÍNH TOÁN
def total_numeric_columns(df_Cognos_cal, df_Looker_cal, columns_to_compare, sales_week_col, start_week=None, end_week=None):
    df_Looker_cal['Source'] = 'Looker'
    df_Cognos_cal['Source'] = 'Cognos'
    combined_data = pd.concat([df_Cognos_cal, df_Looker_cal])
    if start_week and end_week:
        filtered_data = combined_data[(combined_data[sales_week_col] >= start_week) & (combined_data[sales_week_col] <= end_week)]
    else:
        filtered_data = combined_data
    filtered_data[columns_to_compare] = filtered_data[columns_to_compare].round(2)
    result = filtered_data.pivot_table(index=[sales_week_col], columns='Source', values=columns_to_compare, aggfunc='sum')
    for column in columns_to_compare:
        result[f'Compare_{column}_(Cognos <> Looker)'] = result.apply(lambda row: 'Same' if (np.isclose(row[column]['Cognos'], row[column]['Looker'], atol=0.1) == True) else f"{row[column]['Cognos']} <> {row[column]['Looker']}", axis=1)
    result = result.swaplevel(axis=1).sort_index(axis=1, level=[1, 0], ascending=[True, False])
    return result

# ĐỊNH NGHĨA HÀM ĐỂ ĐẶT LẠI TÊN CŨ TRƯỚC KHI XUẤT FILE
# Định nghĩa hàm clean_and_reorder_columns nếu chưa có
def clean_and_reorder_columns(df, suffix_to_remove, new_column_order, new_column_names, output_file):
    df.columns = [col.replace(suffix_to_remove, '') for col in df.columns]
    df = df[new_column_order]
    df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=True)
    df.to_csv(os.path.join(folder_path, output_file), index=False)
    return df

In [42]:
df_cognos = pd.read_excel(r"C:\Users\hoa.nd\Desktop\WAP\compare\report_3\Cognos.xlsx", dtype=str)
df_looker = pd.read_excel(r"C:\Users\hoa.nd\Desktop\WAP\compare\report_3\Looker.xlsx", dtype=str)

In [43]:
df_cognos

,Accounting Year,Accounting Month,Document Type,Activity Year,Activity Month,Location Code,Location Name,Country Code,Country Name,TOFU1 Code,...,TOFU4 Desc,Account Code,Account Name,Cost Code,Cost Name,ACC Estimate Cost (USD),ACC Actual Cost (USD),ACC Accrual Cost (USD),ACT Actual Cost(USD),PL Total Amount(USD)
0,2019,2020/M03,HR,2019,2019/M09,TTPOS,PORT OF SPAIN,TT,TRINIDAD AND TOBAGO,03,...,Port,51102299,OTHER PORT CHARGE,PTSVXX,Charge Other Of Others,-33.2,0,-33.2,NaN,-33.2
1,2019,2020/M03,HR,2019,2019/M08,USOAK,"OAKLAND, CA",US,UNITED STATES,03,...,Port,51102299,OTHER PORT CHARGE,PTSVXX,Charge Other Of Others,-35145.94,-20823.82,-14322.12,NaN,-14322.12
2,2019,2020/M03,HR,2019,2019/M10,USORF,"NORFOLK, VA",US,UNITED STATES,03,...,Port,51102299,OTHER PORT CHARGE,PTSVXX,Charge Other Of Others,-1125,0,-1125,NaN,-1125
3,2019,2020/M03,HR,2019,2019/M09,USORF,"NORFOLK, VA",US,UNITED STATES,03,...,Port,51102299,OTHER PORT CHARGE,PTSVXX,Charge Other Of Others,-900,0,-900,NaN,-900
4,2019,2020/M03,HR,2019,2019/M08,USJAX,"JACKSONVILLE, FL",US,UNITED STATES,03,...,Port,51102202,HARBOR DUE,PTDUHB,Port Charge Harbor Due,-5778.3,0,-5778.3,NaN,-5778.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721932,2023,2023/M11,H5,2023,2023/M11,JPTYO,"TOKYO, TOKYO",JP,JAPAN,03,...,Port,51102202,HARBOR DUE,PTDUHB,Port Charge Harbor Due,NaN,NaN,NaN,379.52,379.52
721933,2023,2023/M11,H5,2023,2023/M11,ITTRS,TRIESTE,IT,ITALY,03,...,Port,51102209,LINE HANDLING,PTSVLI,Port Service Charge Line Handling,NaN,NaN,NaN,440.65,440.65
721934,2023,2023/M11,H5,2023,2023/M10,EGSOK,"SOKHNA, EGYPT",EG,EGYPT,03,...,Port,51102207,PILOTAGE,PTSVSP,Port Service Charge Sea(Straits Pilotage),NaN,NaN,NaN,8152.5,8152.5
721935,2023,2023/M11,H5,2023,2023/M11,AEJEA,JEBEL ALI,AE,UNITED ARAB EMIRATES,03,...,Port,51102205,TONNAGE,PTDUTN,Port Charge Tonnage,NaN,NaN,NaN,9965.18,9965.18


In [44]:
df_looker

,Accounting Fiscal Year,Accounting Calendar Year Month,Document Type Code,Activity Fiscal Year,Activity Calendar Year Month,Location Code,Location Name,Country Code,Country Name,TOFU Group 1 Code,...,TOFU Group 4 Description,Account Code,Account Name,Cost Code,Cost Name,Estimate Cost (USD),Accrual Actual Cost (USD),Accrual Cost (USD),Actual Cost (USD),PL Total Amount (USD)
0,FY2019,2019/M10,HC,FY2019,2019/M10,PECLL,CALLAO,PE,PERU,03,...,Port,51102299,OTHER PORT CHARGE,PTSVGD,Port Service Charge Pier Guard,1889.4,419.87,1469.53,0,1469.53
1,FY2019,2019/M10,HC,FY2018,2019/M02,THBKK,BANGKOK,TH,THAILAND,03,...,Port,51102299,OTHER PORT CHARGE,PTSVHB,Port Service Charge Husbanding Fee,735.84,0,735.84,0,735.84
2,FY2019,2020/M03,HR,FY2019,2019/M05,CNNGB,"NINGBO, ZHEJIANG",CN,CHINA,03,...,Port,51102204,ANCHORAGE,PTDUAN,Port Charge Anchorage,-682.16,-227.39,-454.77,0,-454.77
3,FY2019,2019/M09,HC,FY2018,2019/M01,TTPOS,PORT OF SPAIN,TT,TRINIDAD AND TOBAGO,03,...,Port,51102206,ENTERANCE/CLEARANCE,PTDUBP,Port Charge Berth Priority (Enterance/Clearance),100.02,0,100.02,0,100.02
4,FY2019,2019/M09,HC,FY2019,2019/M08,NGTIN,TIN CAN,NG,NIGERIA,03,...,Port,51102207,PILOTAGE,PTSVSP,Port Service Charge Sea(Straits Pilotage),184.48,0,184.48,0,184.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721932,FY2023,2023/M09,HR,FY2023,2023/M08,USLAX,"LOS ANGELES, CA",US,UNITED STATES,03,...,Port,51102209,LINE HANDLING,PTSVLI,Port Service Charge Line Handling,-43008,-11520,-31488,0,-31488
721933,FY2023,2023/M09,HR,FY2023,2023/M08,INMUN,MUNDRA,IN,INDIA,03,...,Port,51102202,HARBOR DUE,PTDUHB,Port Charge Harbor Due,-1521.44,0,-1521.44,0,-1521.44
721934,FY2023,2023/M07,H5,FY2022,2022/M09,PAROD,RODMAN,PA,PANAMA,03,...,Port,51102204,ANCHORAGE,PTDUAN,Port Charge Anchorage,0,0,0,1685.76,1685.76
721935,FY2023,2023/M09,H5,FY2023,2023/M08,TWKHH,KAOHSIUNG CITY,TW,TAIWAN,03,...,Port,51102299,OTHER PORT CHARGE,PTSVHB,Port Service Charge Husbanding Fee,0,0,0,4483.27,4483.27


In [45]:
COGNOS_TO_LOOKER = {
  'Accounting Year':'Accounting Fiscal Year',
  'Accounting Month':'Accounting Calendar Year Month',
  'Document Type':'Document Type Code',
  'Activity Year':'Activity Fiscal Year',
  'Activity Month':'Activity Calendar Year Month',
  'TOFU1 Code':'TOFU Group 1 Code',
  'TOFU1 Desc':'TOFU Group 1 Description',
  'TOFU2 Code':'TOFU Group 2 Code',
  'TOFU2 Desc':'TOFU Group 2 Description',
  'TOFU3 Code':'TOFU Group 3 Code',
  'TOFU3 Desc':'TOFU Group 3 Description',
  'TOFU4 Code':'TOFU Group 4 Code',
  'TOFU4 Desc':'TOFU Group 4 Description',
  'ACC Estimate Cost (USD)':'Estimate Cost (USD)',
  'ACC Actual Cost (USD)':'Accrual Actual Cost (USD)',
  'ACC Accrual Cost (USD)':'Accrual Cost (USD)',
  'ACT Actual Cost(USD)':'Actual Cost (USD)',
  'PL Total Amount(USD)':'PL Total Amount (USD)'
}
df_cognos.rename(columns=COGNOS_TO_LOOKER, inplace=True)

In [46]:
df_cognos.columns

Index(['Accounting Fiscal Year', 'Accounting Calendar Year Month',
       'Document Type Code', 'Activity Fiscal Year',
       'Activity Calendar Year Month', 'Location Code', 'Location Name',
       'Country Code', 'Country Name', 'TOFU Group 1 Code',
       'TOFU Group 1 Description', 'TOFU Group 2 Code',
       'TOFU Group 2 Description', 'TOFU Group 3 Code',
       'TOFU Group 3 Description', 'TOFU Group 4 Code',
       'TOFU Group 4 Description', 'Account Code', 'Account Name', 'Cost Code',
       'Cost Name', 'Estimate Cost (USD)', 'Accrual Actual Cost (USD)',
       'Accrual Cost (USD)', 'Actual Cost (USD)', 'PL Total Amount (USD)'],
      dtype='object')

In [47]:
df_looker.columns

Index(['Accounting Fiscal Year', 'Accounting Calendar Year Month',
       'Document Type Code', 'Activity Fiscal Year',
       'Activity Calendar Year Month', 'Location Code', 'Location Name',
       'Country Code', 'Country Name', 'TOFU Group 1 Code',
       'TOFU Group 1 Description', 'TOFU Group 2 Code',
       'TOFU Group 2 Description', 'TOFU Group 3 Code',
       'TOFU Group 3 Description', 'TOFU Group 4 Code',
       'TOFU Group 4 Description', 'Account Code', 'Account Name', 'Cost Code',
       'Cost Name', 'Estimate Cost (USD)', 'Accrual Actual Cost (USD)',
       'Accrual Cost (USD)', 'Actual Cost (USD)', 'PL Total Amount (USD)'],
      dtype='object')

In [48]:
# LIST_COLUMN = ['PL Total Amount (USD)','Estimate Cost (USD)']

# compare = datacompy.Compare( df_cognos, df_looker, join_columns=LIST_COLUMN, df1_name='Cognos', df2_name='Looker')
# print(compare.report())

LIST_COLUMN =['Accounting Calendar Year Month',
    'Document Type Code', 'Activity Fiscal Year',
    'Activity Calendar Year Month', 'Location Code', 'Location Name',
    'Country Code', 'Country Name', 'TOFU Group 1 Code',
    'TOFU Group 1 Description', 'TOFU Group 2 Code',
    'TOFU Group 2 Description', 'TOFU Group 3 Code',
    'TOFU Group 3 Description', 'TOFU Group 4 Code',
    'TOFU Group 4 Description', 'Account Code', 'Account Name', 'Cost Code',
    'Cost Name', 'Estimate Cost (USD)', 'Accrual Actual Cost (USD)',
    'Accrual Cost (USD)', 'Actual Cost (USD)']

# combinations_2 = list(combinations(LIST_COLUMN, len(LIST_COLUMN)-1))
# for combo in combinations_2:
#     key=list(combo)
#     if check_columns_combination_uniqueness(df_cognos, df_looker, key):
#         print(key)

# compare= datacompy.Compare(df_cognos, df_looker, on_index=True, df1_name='Cognos', df2_name='Looker')
# print(compare.report())



['Accounting Calendar Year Month', 'Document Type Code', 'Activity Fiscal Year', 'Activity Calendar Year Month', 'Location Code', 'Location Name', 'Country Code', 'Country Name', 'TOFU Group 1 Code', 'TOFU Group 1 Description', 'TOFU Group 2 Code', 'TOFU Group 2 Description', 'TOFU Group 3 Code', 'TOFU Group 3 Description', 'TOFU Group 4 Code', 'TOFU Group 4 Description', 'Account Code', 'Account Name', 'Cost Code', 'Cost Name', 'Estimate Cost (USD)', 'Accrual Actual Cost (USD)', 'Accrual Cost (USD)']
['Accounting Calendar Year Month', 'Document Type Code', 'Activity Fiscal Year', 'Activity Calendar Year Month', 'Location Code', 'Location Name', 'Country Code', 'Country Name', 'TOFU Group 1 Code', 'TOFU Group 1 Description', 'TOFU Group 2 Code', 'TOFU Group 2 Description', 'TOFU Group 3 Code', 'TOFU Group 3 Description', 'TOFU Group 4 Code', 'TOFU Group 4 Description', 'Account Code', 'Account Name', 'Cost Code', 'Cost Name', 'Estimate Cost (USD)', 'Accrual Actual Cost (USD)', 'Actual 

In [ ]:
key= ['Accounting Calendar Year Month', 'Document Type Code', 'Activity Fiscal Year', 'Activity Calendar Year Month', 'Location Code', 'Location Name', 'Country Code', 'Country Name', 'TOFU Group 1 Code', 'TOFU Group 1 Description', 'TOFU Group 2 Code', 'TOFU Group 2 Description', 'TOFU Group 3 Code', 'TOFU Group 3 Description', 'TOFU Group 4 Code', 'TOFU Group 4 Description', 'Account Code', 'Account Name', 'Cost Code', 'Cost Name', 'Estimate Cost (USD)', 'Accrual Actual Cost (USD)', 'Accrual Cost (USD)']
df_merged = pd.merge(df_cognos, df_looker, on=key, how='outer', suffixes=('_cognos', '_looker'))
df_merged